<a href="https://colab.research.google.com/github/Jasper-Hewitt/homework/blob/main/PALM_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# PaLM API: Text Quickstart with Python

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://developers.generativeai.google/tutorials/text_quickstart"><img src="https://developers.generativeai.google/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />View on Generative AI</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/tutorials/text_quickstart.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/generative-ai-docs/blob/main/site/en/tutorials/text_quickstart.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

In this notebook, you'll learn how to get started with the PaLM API, which gives you access to Google's latest large language models. Here, you'll learn how to use the PaLM API's text generation features.

## Setup

**Note**: At this time, the PaLM API is only available in the US.

First, download and install the PaLM API Python library.

In [ ]:
!pip install -q google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.3/113.3 kB 6.6 MB/s eta 0:00:00


In [ ]:
import pprint
import google.generativeai as palm

### Grab an API Key

To get started, you'll need to [create an API key](https://developers.generativeai.google/tutorials/setup).

In [ ]:
palm.configure(api_key='')

## Text generation

Use the `palm.list_models` function to find available models:

In [ ]:
models = [m for m in palm.list_models() if 'generateText' in m.supported_generation_methods]
model = models[0].name
print(model)

models/text-bison-001


Use the `palm.generate_text` method to generate text:

In [ ]:
prompt = """
conduct sentiment analysis on the following three sentences (positive, neutral, negative)
'look at that dog'
'Why would she do that to him?'
'I just won the lottery! Who would've thought?'
"""

completion = palm.generate_text(
    model=model,
    prompt=prompt,
    temperature=0,
    # The maximum length of the response
    max_output_tokens=800,
)

print(completion.result)

1. look at that dog: neutral
2. Why would she do that to him?: negative
3. I just won the lottery! Who would've thought?: positive


## More options

The `palm.generate_text` function has a few other arguments worth mentioning.

### Stop sequences

Use the `stop_sequences` argument to stop generation early.

For example LLM's often make mistakes in arithmetic. You could ask the model to "use a calculator" by putting equations in a `<calc>` tag.

Have the model stop at the closing tag, so you can edit the prompt:

In [ ]:
calc_prompt = f"""
Please solve the following problem.

{prompt}

----------------

Important: Use the calculator for each step.
Don't do the arithmetic in your head.

To use the calculator wrap an equation in <calc> tags like this:

<calc> 3 cats * 2 hats/cat </calc> = 6

----------------

"""

equation=None
while equation is None:
    completion = palm.generate_text(
        model=model,
        prompt=calc_prompt,
        stop_sequences=['</calc>'],
        # The maximum length of the response
        max_output_tokens=800,
    )

    try:
        response, equation = completion.result.split('<calc>', maxsplit=1)
    except Exception:
        continue

In [ ]:
print(response)

Chain-of-thought:

There are three houses, and each house has three cats, so there are 3 houses * 3 cats / house = 9 cats. Each cat has 4 mittens, so the cats need 9 cats * 4 mittens / cat = 36 mittens. Each mitten takes 7m of yarn, so 36 mittens * 7m / mitten = 252m of yarn. Each cat has a hat, and each hat takes 4m of yarn, so 9 cats * 4m / cat = 36m of yarn. So, in total, 36m + 252m = 288m of yarn were needed.

The answer should be 


In [ ]:
print(equation)

 9 cats * 4 mittens / cat 


From there you can calculate the result, and assemble a new prompt for the model to continue from.
For a complete working implementation see the [Text calculator example](../examples/text_calculator.ipynb).

### Candidates

Typically, there's some degree of randomness in the text produced by LLMs. (Read more about why in the [LLM primer](https://developers.generativeai.google/guide/concepts)). That means that when you call the API more than once with the same input, you might get different responses. You can use this feature to your advantage to get alternate model responses.

The `temperature` argument controls the variance of the responses. The `palm.Model` object gives the default value for `temperature` and other parameters.

In [ ]:
models[0]

Model(name='models/text-bison-001', base_model_id='', version='001', display_name='Text Bison', description='Model targeted for text generation.', input_token_limit=8196, output_token_limit=1024, supported_generation_methods=['generateText'], temperature=0.7, top_p=0.95, top_k=40)

The `candidate_count` argument controls the number of responses returned:

In [ ]:
completion = palm.generate_text(
    model=model,
    prompt=prompt,
    # The number of candidates to return
    candidate_count=8,
    # Set the temperature to 1.0 for more variety of responses.
    temperature=1.0,
    max_output_tokens=800,
)

print(completion.result)

In each house there are 3 cats * 4 mittens / cat = 12 mittens. In total there are 3 houses * 12 mittens / house = 36 mittens. In total there are 36 mittens * 7m / mitten = 252m of yarn for the mittens. In total there are 3 houses * 3 cats / house * 1 hat / cat = 9 hats. In total there are 9 hats * 4m / hat = 36m of yarn for the hats. In total there are 36m yarn for the hats + 252m yarn for the mittens = 288m of yarn.
The answer: 288.


When you request multiple candidates the `Completion.result` attribute still just contains the first one. The `Completion.candidates` attribute contains all of them:

In [ ]:
import pprint
pprint.pprint(completion.candidates)

[{'output': 'In each house there are 3 cats * 4 mittens / cat = 12 mittens. In '
            'total there are 3 houses * 12 mittens / house = 36 mittens. In '
            'total there are 36 mittens * 7m / mitten = 252m of yarn for the '
            'mittens. In total there are 3 houses * 3 cats / house * 1 hat / '
            'cat = 9 hats. In total there are 9 hats * 4m / hat = 36m of yarn '
            'for the hats. In total there are 36m yarn for the hats + 252m '
            'yarn for the mittens = 288m of yarn.\n'
            'The answer: 288.',
  'safety_ratings': [{'category': <HarmCategory.HARM_CATEGORY_DEROGATORY: 1>,
                      'probability': <HarmProbability.NEGLIGIBLE: 1>},
                     {'category': <HarmCategory.HARM_CATEGORY_TOXICITY: 2>,
                      'probability': <HarmProbability.NEGLIGIBLE: 1>},
                     {'category': <HarmCategory.HARM_CATEGORY_VIOLENCE: 3>,
                      'probability': <HarmProbability.NEGLIGIBLE: 1>}

So, since you know the answer to this problem, it's easy to check the solve rate:

In [ ]:
import numpy as np
np.mean(['288' in c['output'] for c in completion.candidates])

0.75